In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, Conv2D, LSTM, CuDNNLSTM, Flatten, Dropout, Reshape
from keras import optimizers
from keras import initializers
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
def slidingWindow(sequence, labels, winSize, step, noNull):

    # Verify the inputs
    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")
    if winSize > len(sequence):
        raise Exception("**ERROR** winSize must not be larger than sequence length.")
 
    # number of chunks
    numOfChunks = ((len(sequence)-winSize)//step)+1
 
    # Do the work
    for i in range(0,numOfChunks*step,step):
        segment = sequence[i:i+winSize]
        seg_labels = labels[i:i+winSize]
        if noNull:
            if seg_labels[-1] != 0:
                yield segment, seg_labels
        else:
            yield segment, seg_labels

In [3]:
def segment_data(X_train, y_train, X_val, y_val, X_test, y_test, winSize, step, noNull=False):
    assert len(X_train) == len(y_train)
    assert len(X_val) == len(y_val)
    assert len(X_test) == len(y_test)
    # obtain chunks of data
    train_chunks = slidingWindow(X_train, y_train , winSize, step, noNull)
    val_chunks = slidingWindow(X_val, y_val, winSize, step, noNull)
    test_chunks = slidingWindow(X_test, y_test, winSize, step, noNull)
    
    # segment the data
    train_segments = []
    train_labels = []
    for chunk in train_chunks:
        data = chunk[0]
        labels = chunk[1]
        train_segments.append(data)
        train_labels.append(labels[-1])
        
    val_segments = []
    val_labels = []
    for chunk in val_chunks:
        data = chunk[0]
        labels = chunk[1]
        val_segments.append(data)
        val_labels.append(labels[-1])
    
    test_segments = []
    test_labels = []
    for chunk in test_chunks:
        data = chunk[0]
        labels = chunk[1]
        test_segments.append(data)
        test_labels.append(labels[-1])
        
    return np.array(train_segments), np.array(train_labels), np.array(val_segments), np.array(val_labels), np.array(test_segments), np.array(test_labels)

In [4]:
def prepare_data(train_data, val_data, test_data):
    encoder = OneHotEncoder()
    train_labels = encoder.fit_transform(train_data['labels'].values.reshape(-1,1)).toarray()
    val_labels = encoder.transform(val_data['labels'].values.reshape(-1,1)).toarray()
    test_labels = encoder.transform(test_data['labels'].values.reshape(-1,1)).toarray()
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_data.drop(['labels'], axis=1, inplace=True)
    val_data.drop(['labels'], axis=1, inplace=True)
    test_data.drop(['labels'], axis=1, inplace=True)
    data = pd.concat([train_data,val_data,test_data])
    scaler.fit(data)
    train_data = scaler.transform(train_data)
    val_data = scaler.transform(val_data)
    test_data = scaler.transform(test_data)
    
    return train_data, val_data, test_data, train_labels, val_labels, test_labels

In [5]:
# import train data
adl_1_1 = pd.read_csv("./full_dataset/CIP_interpolation/ADL1Opportunity_taskB2_S1.csv",header=None)
adl_1_2 = pd.read_csv("./full_dataset/CIP_interpolation/ADL2Opportunity_taskB2_S1.csv",header=None)
adl_1_3 = pd.read_csv("./full_dataset/CIP_interpolation/ADL3Opportunity_taskB2_S1.csv",header=None)
adl_1_4 = pd.read_csv("./full_dataset/CIP_interpolation/ADL4Opportunity_taskB2_S1.csv",header=None)
adl_1_5 = pd.read_csv("./full_dataset/CIP_interpolation/ADL5Opportunity_taskB2_S1.csv",header=None)
drill_1 = pd.read_csv("./full_dataset/CIP_interpolation/Drill1Opportunity_taskB2.csv",header=None)
adl_2_1 = pd.read_csv("./full_dataset/CIP_interpolation/ADL1Opportunity_taskB2_S2.csv",header=None)
adl_2_2 = pd.read_csv("./full_dataset/CIP_interpolation/ADL2Opportunity_taskB2_S2.csv",header=None)
drill_2 = pd.read_csv("./full_dataset/CIP_interpolation/Drill2Opportunity_taskB2.csv",header=None)
adl_3_1 = pd.read_csv("./full_dataset/CIP_interpolation/ADL1Opportunity_taskB2_S3.csv",header=None)
adl_3_2 = pd.read_csv("./full_dataset/CIP_interpolation/ADL2Opportunity_taskB2_S3.csv",header=None)
drill_3 = pd.read_csv("./full_dataset/CIP_interpolation/Drill3Opportunity_taskB2.csv",header=None)

# import validation data
adl_2_3 = pd.read_csv("./full_dataset/CIP_interpolation/ADL3Opportunity_taskB2_S2.csv",header=None)
adl_3_3 = pd.read_csv("./full_dataset/CIP_interpolation/ADL3Opportunity_taskB2_S3.csv",header=None)

# import test data
adl_2_4 = pd.read_csv("./full_dataset/CIP_interpolation/ADL4Opportunity_taskB2_S2.csv",header=None)
adl_2_5 = pd.read_csv("./full_dataset/CIP_interpolation/ADL5Opportunity_taskB2_S2.csv",header=None)
adl_3_4 = pd.read_csv("./full_dataset/CIP_interpolation/ADL4Opportunity_taskB2_S3.csv",header=None)
adl_3_5 = pd.read_csv("./full_dataset/CIP_interpolation/ADL5Opportunity_taskB2_S3.csv",header=None)

In [6]:
train_frames = [adl_1_1,adl_1_2,adl_1_3,adl_1_4,adl_1_5,drill_1,adl_2_1,adl_2_2,drill_2,adl_3_1,adl_3_2,drill_3]
val_frames = [adl_2_3,adl_3_3]
test_frames = [adl_2_4,adl_2_5,adl_3_4,adl_3_5]
train_data = pd.concat(train_frames)
val_data = pd.concat(val_frames)
test_data = pd.concat(test_frames)
train_data.rename(columns ={113: 'labels'}, inplace =True)
val_data.rename(columns ={113: 'labels'}, inplace =True)
test_data.rename(columns ={113: 'labels'}, inplace =True)

In [7]:
print("shapes: train {0}, val {1}, test {2}".format(train_data.shape, val_data.shape, test_data.shape))

shapes: train (497014, 114), val (60949, 114), test (118750, 114)


In [8]:
scaled_train, scaled_val, scaled_test, train_labels, val_labels, test_labels = prepare_data(train_data, val_data, test_data)

In [9]:
print(scaled_train.shape)
print(train_labels.shape)

(497014, 113)
(497014, 18)


In [10]:
num_sensors = 113
window_size = 24
step_size = 12
classes = 18

In [11]:
train_segments, train_labels, val_segments, val_labels, test_segments, test_labels = segment_data(scaled_train, train_labels, scaled_val, val_labels,
                                                                                                  scaled_test, test_labels, window_size, step_size)

In [12]:
# reshape input for CNN
reshaped_train = train_segments.reshape(-1, window_size, num_sensors, 1)
reshaped_val = val_segments.reshape(-1, window_size, num_sensors, 1)
reshaped_test = test_segments.reshape(-1, window_size, num_sensors, 1)

In [30]:
model = Sequential()

size_of_kernel = (5,1)
kernel_strides = 1
num_filters = 64
num_lstm_cells = 128
dropout_prob = 0.5
inputshape = (window_size, num_sensors, 1)

In [31]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 activation='relu', input_shape=inputshape,
                 kernel_initializer='glorot_normal', name='1_conv_layer'))

In [32]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 activation='relu',kernel_initializer='glorot_normal',
                 name='2_conv_layer'))

In [33]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 activation='relu',kernel_initializer='glorot_normal',
                 name='3_conv_layer'))

In [34]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 activation='relu',kernel_initializer='glorot_normal',
                 name='4_conv_layer'))

In [35]:
model.add(Reshape((8, num_filters*num_sensors)))

In [36]:
model.add(CuDNNLSTM(num_lstm_cells,kernel_initializer='glorot_normal', return_sequences=True, name='1_lstm_layer'))

model.add(Dropout(dropout_prob, name='2_dropout_layer'))

model.add(CuDNNLSTM(num_lstm_cells,kernel_initializer='glorot_normal',return_sequences=False, name='2_lstm_layer'))

model.add(Dropout(dropout_prob, name='3_dropout_layer'))

model.add(Dense(int(num_lstm_cells/2),kernel_initializer='glorot_normal',
                bias_initializer=initializers.Constant(value=0.1), name='dense_layer'))

model.add(Dense(classes,kernel_initializer='glorot_normal',
                bias_initializer=initializers.Constant(value=0.1),activation='softmax', name='softmax_layer'))

opt = optimizers.RMSprop(lr=0.001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [37]:
for layer in model.layers:
    print(str(layer.name) + ': input shape: ' + str(layer.input_shape) + ' output shape: ' + str(layer.output_shape))

1_conv_layer: input shape: (None, 24, 113, 1) output shape: (None, 20, 113, 64)
2_conv_layer: input shape: (None, 20, 113, 64) output shape: (None, 16, 113, 64)
3_conv_layer: input shape: (None, 16, 113, 64) output shape: (None, 12, 113, 64)
4_conv_layer: input shape: (None, 12, 113, 64) output shape: (None, 8, 113, 64)
reshape_3: input shape: (None, 8, 113, 64) output shape: (None, 8, 7232)
1_lstm_layer: input shape: (None, 8, 7232) output shape: (None, 8, 128)
2_dropout_layer: input shape: (None, 8, 128) output shape: (None, 8, 128)
2_lstm_layer: input shape: (None, 8, 128) output shape: (None, 128)
3_dropout_layer: input shape: (None, 128) output shape: (None, 128)
dense_layer: input shape: (None, 128) output shape: (None, 64)
softmax_layer: input shape: (None, 64) output shape: (None, 18)


In [38]:
batchSize = 100
train_epoches = 50

model.fit(reshaped_train,train_labels,validation_data=(reshaped_val,val_labels),epochs=train_epoches,batch_size=batchSize,verbose=1)

print('Calculating score.. ')
score = model.evaluate(reshaped_test,test_labels,verbose=1)
print(score)
model.save('taskB2_all_Subjects_CNN_LSTM_our_elaboration_full.h5')

Train on 41416 samples, validate on 5078 samples
Epoch 1/50
41416/41416 [==============================] - 50s 1ms/step - loss: 1.0712 - acc: 0.7158 - val_loss: 0.6360 - val_acc: 0.8125
Epoch 2/50
41416/41416 [==============================] - 48s 1ms/step - loss: 0.5836 - acc: 0.7970 - val_loss: 0.5562 - val_acc: 0.8291
Epoch 3/50
41416/41416 [==============================] - 48s 1ms/step - loss: 0.4796 - acc: 0.8291 - val_loss: 0.5296 - val_acc: 0.8529
Epoch 4/50
41416/41416 [==============================] - 48s 1ms/step - loss: 0.4287 - acc: 0.8466 - val_loss: 0.5174 - val_acc: 0.8527
Epoch 5/50
41416/41416 [==============================] - 49s 1ms/step - loss: 0.3850 - acc: 0.8665 - val_loss: 0.4697 - val_acc: 0.8686
Epoch 6/50
41416/41416 [==============================] - 49s 1ms/step - loss: 0.3479 - acc: 0.8805 - val_loss: 0.5123 - val_acc: 0.8600
Epoch 7/50
41416/41416 [==============================] - 49s 1ms/step - loss: 0.3231 - acc: 0.8895 - val_loss: 0.4714 - val_acc:

In [39]:
train_epoches = 20
all_train = np.concatenate((reshaped_train, reshaped_val))
all_labels = np.concatenate((train_labels, val_labels))
model.fit(all_train,all_labels,validation_data=(reshaped_test,test_labels),epochs=train_epoches,batch_size=batchSize,verbose=1)

Train on 46494 samples, validate on 9894 samples
Epoch 1/20
46494/46494 [==============================] - 57s 1ms/step - loss: 0.1188 - acc: 0.9725 - val_loss: 0.4993 - val_acc: 0.8899
Epoch 2/20
46494/46494 [==============================] - 56s 1ms/step - loss: 0.0981 - acc: 0.9741 - val_loss: 0.4215 - val_acc: 0.9061
Epoch 3/20
46494/46494 [==============================] - 56s 1ms/step - loss: 0.0894 - acc: 0.9763 - val_loss: 0.4779 - val_acc: 0.9002
Epoch 4/20
46494/46494 [==============================] - 56s 1ms/step - loss: 0.0814 - acc: 0.9776 - val_loss: 0.4861 - val_acc: 0.9042
Epoch 5/20
46494/46494 [==============================] - 56s 1ms/step - loss: 0.0779 - acc: 0.9786 - val_loss: 0.5358 - val_acc: 0.9038
Epoch 6/20
46494/46494 [==============================] - 56s 1ms/step - loss: 0.0737 - acc: 0.9792 - val_loss: 0.4544 - val_acc: 0.9066
Epoch 7/20
46494/46494 [==============================] - 56s 1ms/step - loss: 0.0673 - acc: 0.9816 - val_loss: 0.5327 - val_acc:

In [40]:
predictions = model.predict(reshaped_test)

In [41]:
# F1-score measure
from sklearn.metrics import f1_score
num_classes = 18
class_predictions = []
class_true = []
tot_labels = 0.0
count = 0.0
for pair in zip(predictions, test_labels):
    class_predictions.append(np.argmax(pair[0]))
    class_true.append(np.argmax(pair[1]))
    if np.argmax(pair[0]) == np.argmax(pair[1]):
        count += 1.0
    tot_labels += 1.0
    
print('Standard accuracy is ' + str(count/tot_labels))    

unique, counts = np.unique(class_true, return_counts=True)
counted_labels = dict(zip(unique, counts))
f1_scores = f1_score(class_predictions, class_true, average=None)

tot_f1_score = 0.0
weights_sum = 0.0
for i in range(num_classes):
    labels_class_i = counted_labels[i]
    weight_i = labels_class_i / tot_labels
    weights_sum += weight_i
    tot_f1_score += f1_scores[i]*weight_i
    print(str(i) + ' ' + str(weight_i) + ' ' + str(f1_scores[i]))

    
print('The weigths sum is ' + str(weights_sum))
print('The computed f1-score is {}'.format(tot_f1_score))
print('The f1-score with sklearn function is {}'.format(f1_score(class_true, class_predictions, average='weighted')))

Standard accuracy is 0.8979179300586214
0 0.832524762482 0.946513446844
1 0.0058621386699 0.571428571429
2 0.00960177885587 0.887640449438
3 0.00606428138266 0.625
4 0.00838892257934 0.88
5 0.0230442692541 0.661654135338
6 0.0161714170204 0.747404844291
7 0.0101071356378 0.616113744076
8 0.00778249444108 0.51497005988
9 0.00394178289873 0.567567567568
10 0.00424499696786 0.394736842105
11 0.0040428542551 0.523076923077
12 0.00262785526582 0.52
13 0.0067717808773 0.651162790698
14 0.00616535273903 0.64406779661
15 0.0100060642814 0.533333333333
16 0.0320396199717 0.615384615385
17 0.0106124924196 0.573170731707
The weigths sum is 1.0
The computed f1-score is 0.8955204164504978
The f1-score with sklearn function is 0.8955204164504977


In [ ]:
pred_df = pd.DataFrame(predictions)
pred_df.to_csv('preds_test.csv', header=False, index=False)

In [ ]:
true_df = pd.DataFrame(testY)
true_df.to_csv('true_test.csv', header=False, index=False)